# Sensitivity
## March Madness Dataset

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import copy
import os
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.stats import skew
from tqdm import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

In [3]:
from pathlib import Path
home = str(Path.home())
home

'/disk/home/amy'

In [4]:
import sys
sys.path.insert(0,"%s/rankability_toolbox_dev"%home)
import pyrankability

In [5]:
sys.path.insert(0,"%s/sensitivity_study/src"%home)
from sensitivity_tests import *
from utilities import *
from base import *

In [6]:
games={}
remaining_games={}
madness_teams={}
all_teams={}
years = ["2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"]
for year in years:
    games[year],remaining_games[year] = read_data('../data/%steams.txt'%year,'../data/%sgames.txt'%year,'../data/%sMadnessTeams.txt'%year)
    madness_teams[year] = list(np.unique(list(games[year].team1_name.loc[games[year].team1_madness == 1]) + list(games[year].team2_name.loc[games[year].team2_madness == 1])))
    all_teams[year] = list(np.unique(list(games[year].team1_name) + list(games[year].team2_name)))
print(year)
games[year]

2018


,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
0,1,16,737011,2017-11-13,1,83,-1,69,Arkansas_St,Abilene_Chr,0,0
1,1,41,737114,2018-02-24,-1,74,1,72,Cent_Arkansas,Abilene_Chr,0,0
3,1,143,737018,2017-11-20,-1,75,1,67,Lipscomb,Abilene_Chr,1,0
4,1,143,737045,2017-12-17,1,67,-1,65,Lipscomb,Abilene_Chr,1,0
5,1,199,737056,2017-12-28,1,77,-1,74,New_Orleans,Abilene_Chr,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5535,351,318,737048,2017-12-20,1,91,-1,74,Utah_St,Youngstown_St,0,0
5536,351,339,737086,2018-01-27,1,85,-1,67,WI_Green_Bay,Youngstown_St,0,0
5537,351,340,737084,2018-01-25,1,66,-1,55,WI_Milwaukee,Youngstown_St,0,0
5538,351,347,737074,2018-01-15,1,77,-1,67,Wright_St,Youngstown_St,1,0


In [7]:
remaining_games[year]

,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
2,1,73,737130,2018-03-12,1,80,-1,73,Drake,Abilene_Chr,0,0
32,2,313,737125,2018-03-07,1,97,-1,90,UNLV,Air_Force,0,0
42,3,79,737126,2018-03-08,0,67,0,58,E_Michigan,Akron,0,0
59,4,137,737128,2018-03-10,0,86,0,63,Kentucky,Alabama,1,1
68,4,326,737135,2018-03-17,0,81,0,58,Villanova,Alabama,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5478,347,287,737133,2018-03-15,0,73,0,47,Tennessee,Wright_St,1,1
5491,348,197,737126,2018-03-08,0,85,0,75,New_Mexico,Wyoming,0,0
5496,349,92,737136,2018-03-18,0,75,0,70,Florida_St,Xavier,1,1
5498,349,234,737127,2018-03-09,0,75,0,72,Providence,Xavier,1,1


In [87]:
import itertools
import joblib

In [15]:
def compute(game_df,team_range,frac,direct_thres,spread_thres,weight_indirect):
    columns = ["frac","direct_thres","spread_thres","weight_indirect"]+team_range
    hillside_columns = ["frac","direct_thres","spread_thres","weight_indirect","details"]

    upper = int(len(game_df)*frac)
    game_df_sample = game_df.iloc[:upper,:]

    map_func = lambda linked: pyrankability.construct.colley_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
    colley_matrix,colley_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
    colley_matrix = colley_matrix.reindex(index=team_range,columns=team_range)
    colley_b = colley_b.reindex(team_range)
    mask = colley_b.isna()
    colley_b = colley_b.loc[~mask]
    colley_matrix = colley_matrix.loc[~mask,~mask]
    inxs = list(np.where(mask)[0])
    ranking1,r1 = pyrankability.construct.ranking_from_matrices(colley_matrix.fillna(0),colley_b,inxs)

    map_func = lambda linked: pyrankability.construct.massey_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
    massey_matrix,massey_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
    massey_matrix = massey_matrix.reindex(index=team_range,columns=team_range)
    massey_b = massey_b.reindex(team_range)
    mask = massey_b.isna()
    massey_b = massey_b.loc[~mask]
    massey_matrix = massey_matrix.loc[~mask,~mask]    
    inxs = list(np.where(mask)[0])    
    ranking2,r2 = pyrankability.construct.ranking_from_matrices(massey_matrix.fillna(0),massey_b,inxs)
    ranking_values1 = [frac,direct_thres,spread_thres,weight_indirect]+list(ranking1)
    ranking_values2 = [frac,direct_thres,spread_thres,weight_indirect]+list(ranking2)
    r_values1 = [frac,direct_thres,spread_thres,weight_indirect]+list(r1)
    r_values2 = [frac,direct_thres,spread_thres,weight_indirect]+list(r2)

    map_func = lambda linked: pyrankability.construct.support_map_vectorized_direct_indirect_weighted(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
    D = pyrankability.construct.V_count_vectorized(game_df_sample,map_func).reindex(index=team_range,columns=team_range)
    k,details = pyrankability.rank.solve(D,method='hillside',lazy=False,cont=True)
    x = pd.DataFrame(details['x'],columns=D.columns,index=D.index)
    c = pd.DataFrame(pyrankability.construct.C_count(D),columns=D.columns,index=D.index)
    P = details['P']
    simple_details = {'k':k,'x':x,'c':c,'P':P,'D':D}
    hillside_ret = pd.Series([frac,direct_thres,spread_thres,weight_indirect,simple_details],index=hillside_columns)
    return pd.Series(ranking_values1,index=columns),pd.Series(ranking_values2,index=columns),pd.Series(r_values1,index=columns),pd.Series(r_values2,index=columns),hillside_ret


In [16]:
def get_game_df(domain_range,year_games):
    # set the team_domain
    team_domain = None
    if domain_range[0] == 'madness':
        team_domain = madness_teams[year]
    elif domain_range[0] == 'all':
        team_domain = all_teams[year]

    game_df = pd.DataFrame({"team1_name":year_games['team1_name'],
                            "team1_score":year_games['points1'],
                            "team1_H_A_N": year_games['H_A_N1'],
                            "team2_name":year_games['team2_name'],
                            "team2_score":year_games['points2'],
                            "team2_H_A_N": year_games['H_A_N1'],
                            "date": year_games['date']
                           }).sort_values(by='date').drop('date',axis=1)
    mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
    game_df = game_df.loc[mask]
    
    return game_df

## What are you fav parameters?

In [17]:
domain_range = ('all','madness')
direct_thres,spread_thres,weight_indirect=2,3,0.5
parameter_string = f"{domain_range},dt={direct_thres},st={spread_thres},iw={weight_indirect}"
parameter_string

"('all', 'madness'),dt=2,st=3,iw=0.5"

## What are you studying?

In [40]:
fracs = [0.7,0.8]

In [41]:
colley_rankings = {}
colley_rs = {}
massey_rankings = {}
massey_rs = {}
hillside_details = {}
outer_keys = list(itertools.product(years,fracs))
for key in tqdm(outer_keys):
    year,frac = key
    # set the team_range
    team_range = None
    if domain_range[1] == 'madness':
        team_range = madness_teams[year]
    elif domain_range[1] == 'all':
        team_range = all_teams[year]
    elif "top" in domain_range:
        team_range = all_teams[year]
    colley_rankings[(year,frac)], colley_rs[(year,frac)], massey_rankings[(year,frac)], massey_rs[(year,frac)], hillside_details[(year,frac)] = compute(get_game_df(domain_range,games[year]),team_range,frac,direct_thres,spread_thres,weight_indirect)


  0%|          | 0/34 [00:00<?, ?it/s]/disk/home/amy/rankability_toolbox_dev/pyrankability/construct.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_df["game"] = list(game_df.index)
/disk/home/amy/rankability_toolbox_dev/pyrankability/construct.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_df["game"] = list(game_df.index)
/disk/home/amy/rankability_toolbox_dev/pyrankability/construct.py:78: RuntimeWarning: invalid value encountered in greater
  mask1 = np.abs(S[:,j]-S[:,i])>threshold 

## Calculate the score you are interested in

In [45]:
import scipy

def calc_corr_set(r1,r2,corr_set):
    return scipy.stats.kendalltau(r1.loc[corr_set],r2.loc[corr_set])[0]

In [57]:
colley_rs[(year,frac1)].loc[corr_set],colley_rs[(year,frac2)].loc[corr_set]

(team2
 Arkansas           8.0
 Auburn            40.0
 Creighton         61.0
 Florida_St        30.0
 New_Mexico_St      9.0
 Oklahoma          46.0
 Seton_Hall        44.0
 UNC_Greensboro    10.0
 Virginia          19.0
 Xavier            50.0
 dtype: float64, team2
 Arkansas          17.0
 Auburn            40.0
 Creighton         31.0
 Florida_St        18.0
 New_Mexico_St     37.0
 Oklahoma          14.0
 Seton_Hall        43.0
 UNC_Greensboro    10.0
 Virginia          19.0
 Xavier            50.0
 dtype: float64)

In [84]:
corr_set

Index(['frac', 'direct_thres', 'spread_thres', 'weight_indirect', 'Auburn',
       'Bucknell', 'Lipscomb', 'Michigan', 'NC_State', 'Radford', 'Syracuse',
       'Tennessee', 'Villanova', 'Virginia'],
      dtype='object')

In [74]:
predictability = pd.DataFrame(columns=["Year","Score","Method","Parameters"])
c = 0
top_n = 10
frac1,frac2 = fracs
for year in tqdm(years):
    perm = massey_rankings[(year,frac1)]
    corr_set = perm.loc[perm<top_n].index
    score = calc_corr_set(massey_rs[(year,frac1)],massey_rs[(year,frac2)],corr_set)
    predictability=predictability.append(pd.Series([year,score,'Massey',parameter_string],index=predictability.columns,name=c)); c+=1 
    
    perm = colley_rankings[(year,frac1)]
    corr_set = perm.loc[perm<top_n].index
    score = calc_corr_set(colley_rs[(year,frac1)],colley_rs[(year,frac2)],corr_set)
    predictability=predictability.append(pd.Series([year,score,'Colley',parameter_string],index=predictability.columns,name=c)); c+=1    


100%|██████████| 17/17 [00:00<00:00, 80.73it/s]


In [75]:
predictability

,Year,Score,Method,Parameters
0,2002,0.872975,Massey,"('all', 'madness'),dt=2,st=3,iw=0.5"
1,2002,0.530395,Colley,"('all', 'madness'),dt=2,st=3,iw=0.5"
2,2003,0.892011,Massey,"('all', 'madness'),dt=2,st=3,iw=0.5"
3,2003,0.419896,Colley,"('all', 'madness'),dt=2,st=3,iw=0.5"
4,2004,0.892583,Massey,"('all', 'madness'),dt=2,st=3,iw=0.5"
5,2004,0.433333,Colley,"('all', 'madness'),dt=2,st=3,iw=0.5"
6,2005,0.901108,Massey,"('all', 'madness'),dt=2,st=3,iw=0.5"
7,2005,0.450549,Colley,"('all', 'madness'),dt=2,st=3,iw=0.5"
8,2006,0.839727,Massey,"('all', 'madness'),dt=2,st=3,iw=0.5"
9,2006,0.530395,Colley,"('all', 'madness'),dt=2,st=3,iw=0.5"


In [76]:
import altair as alt
alt.Chart(predictability).mark_line().encode(
    x='Year',
    y='Score',
    color='Method'
)

alt.Chart(...)

In [77]:
pyrankability.plot.show_hillside?

Signature: pyrankability.plot.show_hillside(V, P0)
Docstring: <no docstring>
File:      ~/rankability_toolbox_dev/pyrankability/plot.py
Type:      function


In [79]:
top_ns = list(range(1,30,1))
graph_df = pd.DataFrame(columns=["Year","k","k_worst","top_n"])
name=0
for year in tqdm(years):
    perm = np.array(hillside_details[(year,frac1)]['details']['P'][0])
    c = hillside_details[(year,frac1)]['details']['c']
    c_reordered = c.iloc[perm,:].iloc[:,perm]
    x_reordered = hillside_details[(year,frac1)]['details']['x'].iloc[perm,:].iloc[:,perm]
    D_reordered = hillside_details[(year,frac1)]['details']['D'].iloc[perm,:].iloc[:,perm]
    k_worst = np.sum(np.triu(c.iloc[perm[::-1],:].iloc[:,perm[::-1]]))
    for top_n in top_ns:
        k = np.sum(np.triu(c.iloc[perm[:top_n],:].iloc[:,perm[:top_n]]))
        #c = c_reordered.iloc[:top_n,:].iloc[:,:top_n]
        #x = np.round(pyrankability.common.threshold_x(x_reordered.iloc[:top_n,:].iloc[:,:top_n].values))
        #k2 = np.sum(np.sum(x*c))
        graph_df = graph_df.append(pd.Series([year,k,k_worst,top_n],name=name,index=graph_df.columns)); name+=1


100%|██████████| 17/17 [00:02<00:00,  6.52it/s]


In [80]:
graph_df['k/k_worst'] = graph_df['k']/graph_df['k_worst']

In [81]:
max_top_n = 12
min_top_n = 4
mask = (graph_df.top_n <= max_top_n) & (graph_df.top_n >= min_top_n)
alt.Chart(graph_df.loc[mask]).mark_line().encode(
    x='top_n',
    y='k/k_worst',
    color='Year'
).interactive()

alt.Chart(...)

In [82]:
slopes = pd.DataFrame(graph_df.loc[mask].groupby('Year').apply(lambda df: np.polyfit(df['top_n'].astype(int), df['k/k_worst'].astype(float),deg=1)[0]))
slopes.columns = [f"Slope over [{min_top_n},{max_top_n}]"]
slopes = slopes.reset_index()
slopes

,Year,"Slope over [4,12]"
0,2002,0.002733
1,2003,0.002538
2,2004,0.001842
3,2005,0.002514
4,2006,0.002703
5,2007,0.002223
6,2008,0.002442
7,2009,0.002510
8,2010,0.002514
9,2011,0.002457


In [83]:
import scipy
for_corr = slopes.set_index('Year').join(predictability.set_index('Year')).reset_index()
for_corr.groupby('Method').apply(lambda df: scipy.stats.pearsonr(df['Slope over [4,12]'],df['Score']))

Method
Colley     (0.17185290727509908, 0.5095550723725694)
Massey    (-0.4222293696861331, 0.09135099113656056)
dtype: object

In [22]:
massey_rankings = {}
colley_rankings = {}
hillside_details = {}
massey_rs = {}
colley_rs = {}

outer_keys = list(itertools.product(domains_ranges,years))
for domain_range,year in tqdm(outer_keys):
    # set the team_domain
    team_domain = None
    if domain_range[0] == 'madness':
        team_domain = madness_teams[year]
    elif domain_range[0] == 'all':
        team_domain = all_teams[year]

    # set the team_range
    team_range = None
    if domain_range[1] == 'madness':
        team_range = madness_teams[year]
    elif domain_range[1] == 'all':
        team_range = all_teams[year]
    elif "top" in domain_range:
        team_range = all_teams[year]

    columns = ["frac","direct_thres","spread_thres","weight_indirect"]+team_range
    massey_rankings[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_rankings[(domain_range,year)] = pd.DataFrame(columns=columns)
    massey_rs[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_rs[(domain_range,year)] = pd.DataFrame(columns=columns)
    hillside_columns = ["frac","direct_thres","spread_thres","weight_indirect","details"]
    hillside_details[(domain_range,year)] = pd.DataFrame(columns=hillside_columns)

    game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                            "team1_score":games[year]['points1'],
                            "team1_H_A_N": games[year]['H_A_N1'],
                            "team2_name":games[year]['team2_name'],
                            "team2_score":games[year]['points2'],
                            "team2_H_A_N": games[year]['H_A_N1'],
                            "date": games[year]['date']
                           }).sort_values(by='date').drop('date',axis=1)
    mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
    game_df = game_df.loc[mask]

    keys = list(itertools.product(fracs,direct_thress,spread_thress,weight_indirects))

    def compute(frac,direct_thres,spread_thres,weight_indirect):
        upper = int(len(game_df)*frac)
        game_df_sample = game_df.iloc[:upper,:]

        map_func = lambda linked: pyrankability.construct.colley_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
        colley_matrix,colley_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
        colley_matrix = colley_matrix.reindex(index=team_range,columns=team_range)
        colley_b = colley_b.reindex(team_range)
        mask = colley_b.isna()
        colley_b = colley_b.loc[~mask]
        colley_matrix = colley_matrix.loc[~mask,~mask]
        inxs = list(np.where(mask)[0])
        ranking1,r1 = pyrankability.construct.ranking_from_matrices(colley_matrix.fillna(0),colley_b,inxs)

        map_func = lambda linked: pyrankability.construct.massey_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
        massey_matrix,massey_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
        massey_matrix = massey_matrix.reindex(index=team_range,columns=team_range)
        massey_b = massey_b.reindex(team_range)
        mask = massey_b.isna()
        massey_b = massey_b.loc[~mask]
        massey_matrix = massey_matrix.loc[~mask,~mask]    
        inxs = list(np.where(mask)[0])    
        ranking2,r2 = pyrankability.construct.ranking_from_matrices(massey_matrix.fillna(0),massey_b,inxs)
        ranking_values1 = [frac,direct_thres,spread_thres,weight_indirect]+list(ranking1)
        ranking_values2 = [frac,direct_thres,spread_thres,weight_indirect]+list(ranking2)
        r_values1 = [frac,direct_thres,spread_thres,weight_indirect]+list(r1)
        r_values2 = [frac,direct_thres,spread_thres,weight_indirect]+list(r2)
        
        map_func = lambda linked: pyrankability.construct.support_map_vectorized_direct_indirect_weighted(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
        D = pyrankability.construct.V_count_vectorized(game_df_sample,map_func).reindex(index=team_range,columns=team_range)
        k,details = pyrankability.rank.solve(D,method='hillside',lazy=False,cont=True)
        x = pd.DataFrame(details['x'],columns=D.columns,index=D.index)
        c = pd.DataFrame(pyrankability.construct.C_count(D),columns=D.columns,index=D.index)
        P = details['P']
        simple_details = {'k':k,'x':x,'c':c,'P':P}
        hillside_ret = pd.Series([frac,direct_thres,spread_thres,weight_indirect,simple_details],index=hillside_columns)
        return pd.Series(ranking_values1,index=columns),pd.Series(ranking_values2,index=columns),pd.Series(r_values1,index=columns),pd.Series(r_values2,index=columns),hillside_ret

    #frac,direct_thres,spread_thres,weight_indirect = keys[0]
    #compute(frac,direct_thres,spread_thres,weight_indirect)
    results = Parallel(n_jobs=-1)(delayed(compute)(frac,direct_thres,spread_thres,weight_indirect) for frac,direct_thres,spread_thres,weight_indirect in keys)

    c = 0
    for i,key in enumerate(keys):
        frac,direct_thres,spread_thres,weight_indirect = key
        massey,colley,massey_r,colley_r,hillside_ret = results[i]
        massey.name = c
        colley.name = c
        colley_r.name=c
        massey_r.name=c
        hillside_ret.name=c
        massey_rankings[(domain_range,year)] = massey_rankings[(domain_range,year)].append(massey)
        colley_rankings[(domain_range,year)] = colley_rankings[(domain_range,year)].append(colley)
        hillside_details[(domain_range,year)] = hillside_details[(domain_range,year)].append(hillside_ret)
        massey_rs[(domain_range,year)] = massey_rs[(domain_range,year)].append(massey_r)
        colley_rs[(domain_range,year)] = colley_rs[(domain_range,year)].append(colley_r)
        c+=1









  0%|          | 0/102 [00:00<?, ?it/s]







  1%|          | 1/102 [01:00<1:41:40, 60.40s/it]







  2%|▏         | 2/102 [01:53<1:37:01, 58.21s/it]







  3%|▎         | 3/102 [02:50<1:35:23, 57.82s/it]







  4%|▍         | 4/102 [03:51<1:36:00, 58.78s/it]







  5%|▍         | 5/102 [04:49<1:34:54, 58.70s/it]







  6%|▌         | 6/102 [05:53<1:36:09, 60.10s/it]







  7%|▋         | 7/102 [06:51<1:34:14, 59.52s/it]







  8%|▊         | 8/102 [07:51<1:33:32, 59.71s/it]







  9%|▉         | 9/102 [08:57<1:35:34, 61.66s/it]







 10%|▉         | 10/102 [10:07<1:38:07, 63.99s/it]







 11%|█         | 11/102 [11:13<1:37:55, 64.57s/it]







 12%|█▏        | 12/102 [12:21<1:38:18, 65.54s/it]







 13%|█▎        | 13/102 [13:29<1:38:23, 66.33s/it]







 14%|█▎        | 14/102 [14:40<1:39:26, 67.80s/it]







 15%|█▍        | 15/102 [15:53<1:40:35, 69.37s/it]







 16%|█▌        | 16/102 [16:50<1:34:04, 65.63s/it]







 17%|█▋        | 17/102 [1

TypeError: can only concatenate list (not "NoneType") to list

In [ ]:
joblib.dump({"colley_rankings":colley_rankings,"massey_rankings":massey_rankings,"massey_rs":massey_rs,"colley_rs":colley_rs},"checkpoint1.joblib.z");